In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np
import pandas as pd
import re 
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", delimiter = "\t")
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", delimiter = "\t")

In [ ]:
train

In [ ]:
test

# *Очистка данных*

#### Для это мы удаляем html теги и непечаnные символы, которые не являются словами. Переводим символы в нижний регистр, разделяем на слова. Удаляем стоп слова. Объединяем слова с добавлением пробела

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []
for i in range(0, num_reviews):
    review_text = BeautifulSoup(train["review"][i]).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]   
    clean_train_reviews.append(" ".join(meaningful_words))

# *Получение векторов признаков из тренировочных данных*

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 10000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

# *Обучение (Random Forest)*

In [ ]:
forest = RandomForestClassifier(n_estimators = 200) 
forest.fit(train_data_features, train["sentiment"])

# *Формирование результатов*

In [ ]:
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    review_text = BeautifulSoup(test["review"][i]).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stopwordss = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stopwordss]  
    clean_test_reviews.append(" ".join(meaningful_words))

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# *Random Forest для предсказаний*

In [ ]:
result = forest.predict(test_data_features)

In [ ]:
output = pd.DataFrame(data = {"id": test["id"], "sentiment": result})
output.to_csv("submission.csv", index=False, quoting = 3)